#### Overview

| Detail Tag | Information |
|------------|-------------|
|Script Name | cdm_load_generic |
|Developed By | Premnath Ghawalkar |
|Input Datasets |<ul><li>u_mfg_production_sap_notification.cdm_config</li></ul>|
|Output Datasets |<ul><li>SAP Notification datasets </li></ul>|
|Input Data Source |<ul><li>SAP Notification</li><li>CDM SAP Notification</li></ul> |
|Output Data Source |<ul><li>Code Orange 2.0 Delta-lake</li><li>Common Data Model</li></ul> |

#### History

| Date | Created/Modified by | Description |
|:----:|--------------|--------|
|23-Feb-2022 | Premnath Ghawalkar |Initial version |


#### Other Details
- This notebook follows based metadata driven data transformation approach. Metadata along with destination operations are defined in "cdm_config" table.

### Import libraries and set parameters

In [0]:
from pyspark.sql.functions import *


## input parameter Dimension table name
dbutils.widgets.text("DestTableName", "vw_ref_ntfy_task_statuscode")
dbutils.widgets.text("DestSchemaName", "u_mfg_production_sap_notification")
dbutils.widgets.text("IsSAP", "1")

p_table_name = dbutils.widgets.get("DestTableName")
p_schema_name = dbutils.widgets.get("DestSchemaName")
p_is_sap = int(dbutils.widgets.get("IsSAP"))

### Set parameters
c_job_loc = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
c_job_name = c_job_loc.rsplit('/',1)[1]

### Logger

In [0]:
%run "../utils/bio_helper_utils"

In [0]:
### set logger
logger = setlogger(app_name=c_job_name, level=logging.INFO)

###  Configure for SAP enterprise mesh node

In [0]:
### set connection string for Enterprise mesh node
if p_is_sap == 1:
  enterprise_mesh = configure_mesh_node(scope_name='copscmfgpus6kv001',
                                      adls_name_secret = 'ent-mesh-adls-name',
                                      tenant_id_secret = 'ent-mesh-tenant-id',
                                      client_id_secret = 'ent-mesh-adls-client-id',
                                      client_key_secret = 'ent-mesh-adls-client-secret')

In [0]:
spark.conf.set("spark.sql.adaptive.enabled",True)
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled",True)
spark.conf.set("spark.sql.inMemoryColumnarStorage.compressed", True)
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

In [0]:
def configure_mesh_node(scope_name: str = None,
                        adls_name_secret: str = None,
                        tenant_id_secret: str = None,
                        client_id_secret: str = None,
                        client_key_secret: str = None):
  
  # get secret from key vault
  ent_adls_name = dbutils.secrets.get(scope=scope_name,key=adls_name_secret)
  tenant_id = dbutils.secrets.get(scope=scope_name,key=tenant_id_secret)
  client_id = dbutils.secrets.get(scope=scope_name,key=client_id_secret)
  client_key = dbutils.secrets.get(scope=scope_name,key=client_key_secret)
  
  # set spark configuration
  spark.conf.set("fs.azure.account.auth.type.{adls}.dfs.core.windows.net".format(adls=ent_adls_name), "OAuth")
  spark.conf.set("fs.azure.account.oauth.provider.type.{adls}.dfs.core.windows.net".format(adls=ent_adls_name),
                 "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
  spark.conf.set("fs.azure.account.oauth2.client.endpoint.{adls}.dfs.core.windows.net".format(adls=ent_adls_name),
                 "https://login.microsoftonline.com/{tenant}/oauth2/token".format(tenant=tenant_id))
  spark.conf.set("fs.azure.account.oauth2.client.id.{adls}.dfs.core.windows.net".format(adls=ent_adls_name),client_id)
  spark.conf.set("fs.azure.account.oauth2.client.secret.{adls_name}.dfs.core.windows.net".format(adls_name=ent_adls_name),client_key)
  return None

In [0]:
enterprise_mesh = configure_mesh_node(scope_name='copscmfgpus6kv001',
                                      adls_name_secret = 'ent-mesh-adls-name',
                                      tenant_id_secret = 'ent-mesh-tenant-id',
                                      client_id_secret = 'ent-mesh-adls-client-id',
                                      client_key_secret = 'ent-mesh-adls-client-secret')

###  Read and gather parameters from config table

In [0]:
config_table_df = spark.sql(""" 
select * from u_mfg_production_sap_notification.cdm_config
where 
active = 1 and
dest_schema_name = '{schema}' and
dest_table_name = '{table_name}'""".format(schema=p_schema_name, table_name=p_table_name))

###  Execute source delta query and merge query

In [0]:
temp_table_name = config_table_df.rdd.collect()
for row in temp_table_name:
  try:
    # create temp view based on temp table name from config table
    src_query = spark.sql("""{query}""".format(query=row["delta_sql_query"]))
    src_query.createOrReplaceTempView(row["temp_table_name"])
    
    # truncate table
    table_action_query = spark.sql("""{query}""".format(query="TRUNCATE table "+ row["dest_schema_name"]+"."+row["dest_table_name"]   ))
    
    # execute merge query
    merge_query = spark.sql("""{merge_query}""".format(merge_query=row["merge_sql_query"]))
    
  except Exception as e:
    msg = 'ERROR: While processing dimension "{dim}", encountered - "{error}" as -'.format(
      dim=str(row["dest_schema_name"]+'.'+row["dest_table_name"]), 
      error=str(e)
    )
    log(msg, logger)